<h1><center>Intro to Data Science Industry Project</h1></center>
<h2><center>Columbia University Professor Rating System</h2></center>

## Webscraping (IDs 20000 to 20100) [Weida]

In [80]:
from bs4 import BeautifulSoup 
import re
from urllib.request import urlopen 
import pandas as pd
from tqdm import tqdm #shows progress

df_200_201 = pd.DataFrame(columns=['review_id', 'review_date' ,'review_text', 'agree_score', 'disagree_score',
                             'funny_score', 'professor_id', 'professor_name'])
for i in tqdm(range(20000, 20100)):
    url = 'http://culpa.info/reviews/{}'.format(i)
    r = urlopen(url).read() 
    soup = BeautifulSoup(r, "lxml") 
    # if review doesn't exist, it redirects to home page, maybe use len() to check if we got redirected
    if len(soup.text) > 15500:
        continue

    professor_id = int(soup.find(href=re.compile('(professor)'))['href'][12:])
    professor_name = soup.find(href=re.compile('(professor)')).text
    review_id = int(soup.find(href=re.compile('(reviews)'))['href'][9:])
    review_text = soup.find(class_='review_content').text
    review_date = pd.Timestamp(soup.find(class_='date').text.strip())
    agree_score = int(re.findall('\d+', soup.find(class_='agree')['value'])[0])
    disagree_score = int(re.findall('\d+', soup.find(class_='disagree')['value'])[0])
    funny_score = int(re.findall('\d+', soup.find(class_='funny')['value'])[0])

    to_append = pd.DataFrame(data=[review_id, review_date, review_text, agree_score, 
                              disagree_score, funny_score, professor_id, professor_name],
                            index=['review_id', 'review_date' ,'review_text', 'agree_score', 
                              'disagree_score', 'funny_score', 'professor_id', 'professor_name']).T
    
    df_200_201 = pd.concat([df_200_201, to_append], ignore_index=True)

100%|██████████| 100/100 [01:45<00:00,  1.05s/it]


#### Check out the scraped data: : First 5 Rows

In [8]:
df_200_201.head()

,review_id,review_date,review_text,agree_score,disagree_score,funny_score,professor_id,professor_name
0,20000,2007-05-18 00:00:00,\nComing off a semester of very little work in...,0,0,0,3281,"Veneziano Broccia, Lillyrose"
1,20001,2007-05-18 00:00:00,\nI had Professor Robbins for a lecture in the...,4,2,0,744,"Robbins, Bruce"
2,20002,2007-05-18 00:00:00,\nIt's a little hard for me to criticize Profe...,5,2,0,115,"Sacks, Richard"
3,20003,2007-05-18 00:00:00,\nThis class was a disaster. Professor Daniel ...,4,2,0,456,"Daniel, Valentine"
4,20004,2007-05-19 00:00:00,\nLovely class and professor. The class was mo...,3,3,0,2371,"Radwan, Noha"


## Webscraping (IDs 70000 to 80000) [Weida v2]

In [ ]:
df = pd.DataFrame(columns=['review_id', 'review_date' ,'review_text', 'agree_score', 'disagree_score',
                             'funny_score', 'professor_id', 'professor_name', 'course_id', 'course_name'])
for i in tqdm(range(70000, 80000)):
    url = 'http://culpa.info/reviews/{}'.format(i)
    r = urlopen(url).read() 
    soup = BeautifulSoup(r, "lxml") 
    # if review doesn't exist, it redirects to home page, maybe use len() to check if we got redirected
    if len(soup.text) > 15500:
        continue
    
    professor = soup.find(href=re.compile('(professor)'))
    if professor == None:
        professor_id = None
        professor_name = None
    else:
        professor_id = professor['href'][12:]
        professor_name = professor.text
    
    courses = soup.find_all(href=re.compile('(courses)'))
    if courses == None:
        course_id = None
        course_name = None
    else:
        course_id = courses[-(len(courses) - 7)]['href'][9:]
        course_name = courses[-(len(courses) - 7)].text

    review_id = soup.find(href=re.compile('(reviews)'))['href'][9:]
    review_text = soup.find(class_='review_content').text
    review_date = pd.Timestamp(soup.find(class_='date').text.strip())
    agree_score = re.findall('\d+', soup.find(class_='agree')['value'])[0]
    disagree_score = re.findall('\d+', soup.find(class_='disagree')['value'])[0]
    funny_score = re.findall('\d+', soup.find(class_='funny')['value'])[0]

    to_append = pd.DataFrame(data=[review_id, review_date, review_text, agree_score, 
                                   disagree_score, funny_score, professor_id, professor_name,
                                   course_id, course_name],
                            index=['review_id', 'review_date' ,'review_text', 'agree_score', 
                                   'disagree_score', 'funny_score', 'professor_id', 
                                   'professor_name', 'course_id', 'course_name']).T
    
    df = pd.concat([df, to_append], ignore_index=True)

In [ ]:
'''from bs4 import BeautifulSoup 
import re
from urllib.request import urlopen 
import pandas as pd
from tqdm import tqdm

df = pd.DataFrame(columns=['review_id', 'review_date' ,'review_text', 'agree_score', 'disagree_score',
                             'funny_score', 'professor_id', 'professor_name'])
for i in tqdm(range(70000, 80000)):
    url = 'http://culpa.info/reviews/{}'.format(i)
    r = urlopen(url).read() 
    soup = BeautifulSoup(r, "lxml") 
    # if review doesn't exist, it redirects to home page, maybe use len() to check if we got redirected
    if len(soup.text) > 15500:
        continue

    professor_id = soup.find(href=re.compile('(professor)'))['href'][12:]
    professor_name = soup.find(href=re.compile('(professor)')).text
    review_id = soup.find(href=re.compile('(reviews)'))['href'][9:]
    review_text = soup.find(class_='review_content').text
    review_date = pd.Timestamp(soup.find(class_='date').text.strip())
    agree_score = re.findall('\d+', soup.find(class_='agree')['value'])[0]
    disagree_score = re.findall('\d+', soup.find(class_='disagree')['value'])[0]
    funny_score = re.findall('\d+', soup.find(class_='funny')['value'])[0]

    to_append = pd.DataFrame(data=[review_id, review_date, review_text, agree_score, 
                              disagree_score, funny_score, professor_id, professor_name],
                            index=['review_id', 'review_date' ,'review_text', 'agree_score', 
                              'disagree_score', 'funny_score', 'professor_id', 'professor_name']).T
    
    df = pd.concat([df, to_append], ignore_index=True)
    '''

In [ ]:
df.professor_id = df.professor_id.apply(int)

In [ ]:
df

## Webscraping (IDs 20000 to 20200)

In [81]:
df_200_202 = pd.DataFrame(columns=['review_id', 'review_date' ,'review_text', 'agree_score', 'disagree_score',
                             'funny_score', 'professor_id', 'professor_name'])
for i in tqdm(range(20000, 20200)):
    url = 'http://culpa.info/reviews/{}'.format(i)
    r = urlopen(url).read() 
    soup = BeautifulSoup(r, "lxml") 
    # if review doesn't exist, it redirects to home page, maybe use len() to check if we got redirected
    if len(soup.text) > 15500:
        continue

    professor_id = int(soup.find(href=re.compile('(professor)'))['href'][12:])
    professor_name = soup.find(href=re.compile('(professor)')).text
    review_id = int(soup.find(href=re.compile('(reviews)'))['href'][9:])
    review_text = soup.find(class_='review_content').text
    review_date = pd.Timestamp(soup.find(class_='date').text.strip())
    agree_score = int(re.findall('\d+', soup.find(class_='agree')['value'])[0])
    disagree_score = int(re.findall('\d+', soup.find(class_='disagree')['value'])[0])
    funny_score = int(re.findall('\d+', soup.find(class_='funny')['value'])[0])

    to_append = pd.DataFrame(data=[review_id, review_date, review_text, agree_score, 
                              disagree_score, funny_score, professor_id, professor_name],
                            index=['review_id', 'review_date' ,'review_text', 'agree_score', 
                              'disagree_score', 'funny_score', 'professor_id', 'professor_name']).T
    
    df_200_202 = pd.concat([df_200_202, to_append], ignore_index=True)

100%|██████████| 200/200 [00:33<00:00,  6.01it/s]


#### Check out the scraped Data: First 5 Rows

In [86]:
df_200_202.head()

,review_id,review_date,review_text,agree_score,disagree_score,funny_score,professor_id,professor_name
0,20000,2007-05-18 00:00:00,\nComing off a semester of very little work in...,0,0,0,3281,"Veneziano Broccia, Lillyrose"
1,20001,2007-05-18 00:00:00,\nI had Professor Robbins for a lecture in the...,4,2,0,744,"Robbins, Bruce"
2,20002,2007-05-18 00:00:00,\nIt's a little hard for me to criticize Profe...,5,2,0,115,"Sacks, Richard"
3,20003,2007-05-18 00:00:00,\nThis class was a disaster. Professor Daniel ...,4,2,0,456,"Daniel, Valentine"
4,20004,2007-05-19 00:00:00,\nLovely class and professor. The class was mo...,3,3,0,2371,"Radwan, Noha"


## Trying Natural Language Toolkit

In [ ]:
import nltk
#nltk.download() # only need to do this once

In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

/Users/pranjalbajaj/anaconda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


#### Take a subset of the data (df_200_202) for NLTK analysis

In [87]:
subset = df_200_202[:20] # subset of the first 10 reviews
reviews = subset['review_text'] # just the review column

In [88]:
all_pred_sentiments = []

for review in tqdm(reviews): # for each review, 
    pred_sentiment = sid.polarity_scores(review) # calculate the sentiment scores
    all_pred_sentiments.append(pred_sentiment) # and append them to a list

100%|██████████| 20/20 [00:00<00:00, 373.21it/s]


In [89]:
subset = pd.concat([subset, pd.DataFrame(all_pred_sentiments)], axis=1) # add these new columns to our dataframe!

In [90]:
subset.head()

,review_id,review_date,review_text,agree_score,disagree_score,funny_score,professor_id,professor_name,compound,neg,neu,pos
0,20000,2007-05-18 00:00:00,\nComing off a semester of very little work in...,0,0,0,3281,"Veneziano Broccia, Lillyrose",0.9949,0.028,0.781,0.190
1,20001,2007-05-18 00:00:00,\nI had Professor Robbins for a lecture in the...,4,2,0,744,"Robbins, Bruce",0.9966,0.033,0.735,0.231
2,20002,2007-05-18 00:00:00,\nIt's a little hard for me to criticize Profe...,5,2,0,115,"Sacks, Richard",0.9941,0.040,0.782,0.177
3,20003,2007-05-18 00:00:00,\nThis class was a disaster. Professor Daniel ...,4,2,0,456,"Daniel, Valentine",0.9136,0.095,0.754,0.151
4,20004,2007-05-19 00:00:00,\nLovely class and professor. The class was mo...,3,3,0,2371,"Radwan, Noha",0.8764,0.015,0.862,0.123


#### Add weights for Agree/Disagree scores and append *Weighted Scores* to the DataFrame

Formula to calculate weights:

In [104]:
weight = subset.loc[1,'agree_score'] * subset.loc[1,'agree_score'] + subset.loc[1,'disagree_score']

Formula to calculate weights while iterating (Sorry, can't get it to function -- brain dead right now haha :P)
#### Can anyone fix this?

In [136]:
weighted_score = pd.DataFrame({'weighted_score' : []}) #empty pandas column

for i in tqdm(range(len(subset.index))): #iterate over length of dataframe (i.e. subset.index)
    denominator = subset.loc[i,'agree_score'] + subset.loc[i,'disagree_score']
    
    if denominator != 0: #to avoid math error 
        weights = subset.loc[i,'agree_score'] / denominator
        w_score = subset.loc[i,'compound'] * weight #calculate weighted score (w_score)
        weighted_score.append(w_score) #append it to the empty pandas column

subset = pd.concat([subset, weighted_score], ignore_index=True) #add to the our dataframe, subset


  0%|          | 0/20 [00:00<?, ?it/s]


TypeError: cannot concatenate a non-NDFrame object